In [1]:
import numpy as np
import pandas as pd
import xarray as xr

import matplotlib.pyplot as plt


In [2]:
# targets
delta_p = -10  # regulation precision [dbar]
delta_m = 0.3 # surface mass excess [kg]
v_down = 0.5 # initial descent speed [m/s]
p = 200 # target depth 5dbar]

In [3]:
# platform parameters
a = 0.08
h = 1.
# external module
A = 0.15
H = 0.1
V_f_s = np.pi*a**2/2 * h + np.pi*(A**2 - a**2)/2 * H
# compressibility
gamma = 2e-6 # 1/dbar
# drag
c1 = 1

# water
rho_w_0 = 1030 # [kg/m^3]
delta_rho_w = 0.  # [kg/m^3]
gamma_w = 6e-6 # 1/dbar
drho_w_dp = 2*gamma_w*rho_w_0  # 1/dbar

In [4]:
# beads
rho_b = 2*rho_w_0

# dead weights
rho_0 = 2*rho_w_0
rho_1 = 2*rho_w_0

In [5]:
T = 10 # days
rate_b = 1/10 # beads per minutes

N_b = int(T*24*60*rate_b)
print(f"{N_b} beads required for {T} days deployment and rate of release of {rate_b}")

1440 beads required for 10 days deployment and rate of release of 0.1


Unkown variables: $m$, $m_b$, $m_1$

In [6]:
rho_w = lambda p: rho_w_0 + delta_rho_w + drho_w_dp*p
V_f = lambda p: V_f_s*(1 - gamma*p)

In [7]:
# state vector: m, m_1, m_b

M_equ = np.array([1, (rho_1-rho_w(p))/rho_1, N_b*(rho_b-rho_w(p))/rho_b])
b_equ = rho_w(p)*V_f(p)

M_reg = np.array([0, -delta_p*drho_w_dp/rho_1, -(rho_b-rho_w(p))/rho_b -N_b*delta_p*drho_w_dp/rho_b])
b_reg = delta_p * V_f(p) * (drho_w_dp - rho_w(p)*gamma)

M_sur = np.array([1, 0, N_b*(rho_b-rho_w(0))/rho_b])
b_sur = rho_w(0)*V_f(0) - delta_m

In [8]:
M = np.vstack((M_equ, M_reg, M_sur))
b = np.array([b_equ, b_reg, b_sur])

In [9]:
m, m_1, m_b = np.linalg.solve(M, b)

In [10]:
print(f" m={m:.2f}kg , m_b={m_b*1e3:.1f}g, m_1={m_1*1e3:.1f}g")

 m=10.33kg , m_b=3.2g, m_1=664.6g


In [11]:
v_b = m_b/rho_b
r_b = (3/4*v_b)**(1/3)
print(f"bead radius = {r_b*1e3}mm")

bead radius = 10.562138955457888mm


In [12]:
rho_w(0), rho_w(p)

(1030.0, 1032.472)

In [14]:
V_f(0), V_f(p)

(0.012582078577627121, 0.01257704574619607)

In [15]:
N_b*m_b

4.660409505348723

In [ ]:
compute m_0

In [13]:
# initial guess
m = rho_w_0*V_f_s
v_b = 1e-7
v_0 = 100*1e-6
v_1 = 100*1e-6

# derived variables

m_m0 = m + v_1*rho_1 + N_b*rho_b*v_b
V_m0 = V_f_s*(1-gamma*p) + v_1 + N_b*v_b
rho_f_m0 = m_m0/V_m0

# (150)
v_b = (drho_w_dz / rho_f_m0 - V_f_s*gamma) * delta_p * V_m0 / (1 - rho_b/rho_f_m0)
m_b = rho_b*v_b

# (145), (161), assumes condition controlled by surface emergence
delta_m_p = (rho_w_0 + delta_rho_w) * V_f_s * (gamma_w - gamma) * p
m_1 = (delta_m + delta_m_p) * rho_1/(rho_1 - rho_w_0 - delta_rho_w)
# leads to v_1 update
v_1 = m_1/rho_1

# (143)
m_f = (rho_w_0+delta_rho_w)*(V_f_s + N_b*v_b) - N_b*m_b
m = m_f + delta_m_p - (rho_1 - rho_w_0 - delta_rho_w) * v_1

# (134), (136)
cd = c1/2/h * abs(v_down)
rho_f = rho_w_0/(1+cd*abs(v_down)) 
m_0 = rho_0 * (rho_f * V_m0 - m_m0)/(rho_0 - rho_f)


In [ ]:
m_guess = 